In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import pickle


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# dfの各列の型を設定しメモリ軽減
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## データの標準化

In [ ]:
# read data
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
test = pd.read_csv('/kaggle/input/jane-street-market-prediction/example_test.csv')

In [ ]:
# 欠損値の補完(前の値で補完する)
train.fillna(method = 'ffill', inplace=True) 
train.dropna(inplace=True)

In [ ]:
# respについて
resp_params = (train['resp'].mean(), train['resp'].std())
resp_standardized = ((train['resp'] - resp_params[0])/resp_params[1]).values
resp_info = (resp_params, resp_standardized)

In [ ]:
# 基準化
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(train[test.columns])
Z = sc.transform(train[test.columns])
train = pd.DataFrame(Z, columns=test.columns) # dfの整形
train = reduce_mem_usage(train) # メモリ対策

In [ ]:
# 保存
import pickle
train.to_pickle('/kaggle/working/train_standardized_without_null.pickle')
with open('/kaggle/working/SC.pickle', 'wb') as f:
    pickle.dump(sc, f)
with open('/kaggle/working/resp_info.pickle', 'wb') as f:
    pickle.dump(resp_info, f)

## 主成分分析(PCA)

In [ ]:
from sklearn.decomposition import PCA 
import matplotlib.pyplot as plt

# 特徴量のみにする
X = train.drop(['weight', 'date', 'ts_id'],axis=1).values

# 主成分分析
pca = PCA()
pca.fit(X)
# データを主成分空間に写像(主成分スコア)
score = pca.transform(X)

# 累積寄与率を図示する
plt.plot([0] + list( np.cumsum(pca.explained_variance_ratio_)), "-o")
plt.xlabel("Number of principal components")
plt.ylabel("Cumulative contribution rate")
plt.grid()
plt.show()

> 上図から概ね第30主成分程度で特徴量全体の90%を表現できていることがわかる。調べてみたら第16主成分で80%、第20主成分で85%、第28主成分目で90%を上回っていた。

## リターンと各主成分の関係

In [ ]:
import seaborn as sns
# respと主成分スコアの関係
# df作成
target = pd.DataFrame(np.concatenate([resp_info[1][:, np.newaxis], score[:, :16]], axis=1))
target.columns = pd.Index(['resp'] + ['PC{}'.format(i+1) for i in range(16)])
# ヒートマップ作成
sns.heatmap(target.corr())

各主成分やrespの間に相関は低いことがわかる。

## 主成分回帰モデル(PCRモデル)

In [ ]:
import statsmodels.api as sm

# 'weight'を追加
target = pd.concat([target, train['weight']], axis=1).copy()
target.head()

# PCR
x = target.drop(['resp'], axis=1)
x = sm.add_constant(x)
model = sm.OLS(target['resp'], x)
result = model.fit()
print(result.summary())

# グラフを書く
plt.plot(target['resp'], label='resp', linestyle='--')
result.fittedvalues.plot(label='fitted', style=':')
plt.legend()

全期間で線形回帰を行った結果、決定係数は0.002と酷い。全期間のデータを使うのは賢明ではなさそう。周期性などを考慮したほうが良いのかもしれない。次に、最初の100個を対象に最適化を行った結果は以下の通りである。

In [ ]:

# PCR
x = target.drop(['resp'], axis=1)
x = sm.add_constant(x)
model = sm.OLS(target['resp'][:100], x[:100])
result = model.fit()
print(result.summary())

# グラフを書く
plt.plot(target['resp'][:100], label='resp', linestyle='--')
result.fittedvalues.plot(label='fitted', style=':')
plt.legend()

そこそこフィットしている様子が見られる。データを少なくすればフィットするようである。期間を複数に分け、各期間ごとで最適なモデルを採用するようなものにすると良いのかもしれない。分析に入る前に、リターンの特徴(周期性など)を最初に見るべきだった。

## クラスタリング(K-means)

In [ ]:
from sklearn.cluster import KMeans # K-means
kmeans_model = KMeans(n_clusters=5, random_state=0).fit(target.iloc[:, 1:]) # resp以外
km_result = pd.concat([target, pd.DataFrame(kmeans_model.labels_, columns=['cluster'])],axis=1)
km_result.groupby('cluster').describe()['resp']

In [ ]:
#respを正負に変換し、クラスタごとにカウントしてみる。
km_result['resp_pn'] = km_result['resp'].apply(lambda x:'p' if x>0 else 'n')
km_result.groupby(['cluster', 'resp_pn']).count()['resp'].plot.bar(color=['blue', 'red'])

カウントベースでみると、クラスター1と3ではマイナスの方が数が多いことがわかる。クラスター1については、平均もマイナスであり、マイナスの特性を持ちそうだ。

In [ ]:
# クラスター数:3
kmeans_model = KMeans(n_clusters=3, random_state=0).fit(target.iloc[:, 1:]) # resp以外
km_result = pd.concat([target, pd.DataFrame(kmeans_model.labels_, columns=['cluster'])],axis=1)
km_result.groupby('cluster').describe()['resp']
km_result['resp_pn'] = km_result['resp'].apply(lambda x:'p' if x>0 else 'n')
km_result.groupby(['cluster', 'resp_pn']).count()['resp'].plot.bar(color=['blue', 'red'])

In [ ]:
# クラスター数:10
kmeans_model = KMeans(n_clusters=3, random_state=0).fit(target.iloc[:, 1:]) # resp以外
km_result = pd.concat([target, pd.DataFrame(kmeans_model.labels_, columns=['cluster'])],axis=1)
km_result.groupby('cluster').describe()['resp']
km_result['resp_pn'] = km_result['resp'].apply(lambda x:'p' if x>0 else 'n')
km_result.groupby(['cluster', 'resp_pn']).count()['resp'].plot.bar(color=['blue', 'red'])

クラスターで分けることでマイナスの特徴を捉えることができている可能性がある。また、クラスター数は、３程度でも十分と思われる。ひとまず、今回の結果を踏まえると、モデル開発においてクラスター番号を入れてみる価値はありそうだ。ただ、ポジティブの特徴について見ると、クラスター数5の(2,p)が捉えることができている可能性があるのを踏まえると、クラスター数は5としてみようと思う。

## 主成分分析＋クラスタリング

In [ ]:
# PCR + Clustering
x = km_result.drop(['resp', 'resp_pn'], axis=1)
x = sm.add_constant(x)
model = sm.OLS(km_result['resp'][:100], x[:100])
result = model.fit()
print(result.summary())

あまり改善はされていない。変数が一つ増えたことで、補正Ｒ２は低下している。
そもそも主成分分析を行い、次元数を縮小させることに価値があるのか確認のため、元の特徴量(feature_X)で全期間に対して線形回帰を行ってみた。